How's it look?

Now with **Mixture model**!

We have orders m111, m113, m114, m115 complete.  Let's plot all of the results on the same figure.

In [ ]:
n_o = 23
go = np.arange(72, 94+1)

In [ ]:
n_s = 200
n_ws = 40
n_par = 14+1
wss = np.zeros((n_s*n_ws, n_par, n_o))

In [ ]:
for m in go:
    ws = np.load("../sf/m{:03d}/output/mix_emcee/run01/emcee_chain.npy".format(m))
    burned = ws[:, -1.0*n_s:,:]
    xs, ys, zs = burned.shape
    fc = burned.reshape(xs*ys, zs)
    wss[:, :-1, m-72] = fc
    wss[:, -1, m-72] = 10**fc[:,7] / ( 10**fc[:,5] + 10**fc[:,7] )

I'm pretty sure that what I'm about to do is *not allowed* by probability theory.  
Specifically, I'm "adding probabilities" an operation that is not principled... I think.  
It would be great to ask a statistician about this!
In any case, the result can be considered a consistency check, or an order-weighted average joint probability.

In [ ]:
bb = np.array([])
tt = np.array([])

In [ ]:
for m in np.arange(n_o):
    bb = np.append(bb, wss[:,-1, m], axis=0)
    tt = np.append(tt, wss[:,6, m], axis=0)

In [ ]:
ax = sns.jointplot(bb, tt, kind='scatter', alpha=0.005)
ax.ax_joint.set_ylabel('Star spot $T_{\mathrm{eff}}$')
ax.ax_joint.set_xlabel('Coverage fraction $f$')
#plt.savefig('Star_spot_measurement.pdf', bbox_inches='tight')

This takes a while:

In [ ]:
ax = sns.jointplot(bb, tt, kind='kde')
ax.ax_joint.set_ylabel('Star spot $T_{\mathrm{eff}}$')
ax.ax_joint.set_xlabel('Coverage fraction $f$')

# Make a pandas dataframe of the average values from the inference

In [ ]:
import pandas as pd

In [ ]:
new_df = pd.read_csv('../data/analysis/IGRINS_mix_emcee_last200.csv')

Copy the metadata columns from the old dataframe:

What columns do we want?

- All the same as the last array: median and 2 sigma ranges. (6+6)*3 = 36 
- Teff2 and Omega2 and 2 sigma ranges. (1+1)*3 = 6
- Conversion of Omega2 and Omega1 into fill factor and 2 sigma ranges. (1)*3 = 3
- Metadata about the number of samples and the burn-in assumed to compute the above values.
    - We can get more metadata from the `sacct` log from slurm

In [ ]:
for m in new_df.m_val:
    ind = new_df.index[new_df.m_val == m]
    try:
        ws = np.load("../sf/m{:03d}/output/mix_emcee/run01/emcee_chain.npy".format(m))
        new_df.set_value(ind, 'inference', True)
    except:
        new_df.set_value(ind, 'inference', False)

In [ ]:
cs = new_df.columns[7:-5:3]
bases = cs.str[:-4]

In [ ]:
bases

In [ ]:
base_inds = np.hstack([np.arange(6), np.arange(8, 14), [6, 7]])

In [ ]:
suffixes = ['_05p', '_50p','_95p']

In [ ]:
base_dict = {key:val for key, val in zip(bases, base_inds)}
suffix_dict = {key:val for key, val in zip(suffixes, [5, 50, 95])}

In [ ]:
for m in new_df.m_val[new_df.inference]:
    ind = new_df.index[new_df.m_val == m]
    ws = np.load("../sf/m{:03d}/output/mix_emcee/run01/emcee_chain.npy".format(m))
    burned = ws[:, -1*n_s:,:]
    xs, ys, zs = burned.shape
    fc = burned.reshape(xs*ys, zs)
    bb = 10**fc[:,7] / ( 10**fc[:,5] + 10**fc[:,7] )
    for suffix in suffixes:
        new_df.set_value(ind, 'ff'+suffix, np.percentile(bb, suffix_dict[suffix]))
        for base in bases[:-1]:
            new_df.set_value(ind, base+suffix, np.percentile(fc[:, base_dict[base]], suffix_dict[suffix]))

In [ ]:
new_df.tail()

In [ ]:
new_df.to_csv('../data/analysis/IGRINS_mix_emcee_last200.csv', index=False)

The end.